#3.2 Prompt Templates

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}",
                                        )
prompt = template.format(country_a = "Mexico", country_b = "Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content ="You are a geography expert. And you only reply in Italian.",
                  ),
    AIMessage(content = "Ciao, mi chiamo Paolo!"
    ),
    HumanMessage(content = "What is the distance between Mexico and Thailand. Also, what is your name?"
                 ),
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono un esperto di geografia. Avete altre domande per me?')

In [19]:
template = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human",
     "What is the distance between {country_a} and {country_b}. Also, what is your name?"
     ),
    ]
    )

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

#3.3 OutputParser and LCEL

In [23]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text): #strip : 앞.뒤 공백 자름
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello,how,are,you")
p.parse("Hello,  how,  are,  you")

['Hello', 'how', 'are', 'you']

In [31]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "What are the planets?"
)

chat.predict_messages(prompt)


AIMessage(content='mercury, venus, earth, mars, jupiter, saturn, uranus, neptune')

In [35]:

prompt = template.format_messages(
    max_items = 10,
    question = "What are the colors?"
)

result = chat.predict_messages(prompt)

In [36]:
p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [37]:
#langchain -> template | chatModel | outputParser 이면 충분

chain = template | chat | CommaOutputParser()  #긴 코드가 한 줄로 생략됨

"""example
chain_one = template_1 | chat | outputparser_one()
chain_two = template_2 | chat | outputparser_two()
all = chain_one | chain_two | outputparser()
"""

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

#3.4 Chaining Chains

In [47]:
#components of langchain : Prompt(dict), Retriever(str), LLM/ChatModel(str or list or promptvalue), 
# Tool(str or dict), OutputParser(output of LLM/ChatModel)

#output type : LLM(str), ChatModel(ChatMessage), Prompt(PromptValue)

#################Let's start from zero####################

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming = True, callbacks=[StreamingStdOutCallbackHandler()]) #진행 상황을 live로 볼 수 있다

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat # 레시피 전해주는 chef

In [42]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

In [48]:
veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain #{recipe}가 veg_chain의 input으로 와야 하므로

final_chain.invoke({
    "cuisine":"indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and easy recipe for Chicken Tikka Masala.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp paprika
- 1 tsp ground turmeric
- 1 tsp garam masala
- 1 tsp ground coriander
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Place the marinated chick

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare the tofu or paneer as a replacement for the chicken in this recipe:\n\n**Tofu:**\n1. Use firm or extra-firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-20 minutes.\n2. Cut the pressed tofu into bite-sized cubes and proceed with marinating it as you would with the chicken in the recipe.\n3. Bake the marinated tofu in the oven at 400°F (200°C) for about 20-25 minutes or until it is slightly crispy on the outside.\n\n**Paneer:**\n1. If you prefer paneer over tofu, you can use store-bought or homemade paneer for this recipe.\n2. Cut the paneer into bite-sized cubes and lightly fry them in a pan until they develop a golden crust on the outside. This step helps the paneer hold its sha